## torchvision.models.segmentationを使う

In [4]:
import os
from tifffile import TiffFile
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torchvision.models import vgg16_bn
import torch.optim as optim
from tqdm import tqdm
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
# from torchsummary import summary

from torchvision.io.image import read_image
from torchvision.models.segmentation import fcn_resnet50, FCN_ResNet50_Weights
from torchvision.transforms.functional import to_pil_image

In [5]:
data_dir = '../platelet_data'

with TiffFile(os.path.join(data_dir, 'train-images.tif')) as tif:
    train_img = tif.asarray()
    
with TiffFile(os.path.join(data_dir, 'train-labels.tif')) as tif:
    train_label = tif.asarray()
    
with TiffFile(os.path.join(data_dir, 'eval-images.tif')) as tif:
    eval_img = tif.asarray()

with TiffFile(os.path.join(data_dir, 'eval-labels.tif')) as tif:
    eval_label = tif.asarray()
    
with TiffFile(os.path.join(data_dir, 'test-images.tif')) as tif:
    test_img = tif.asarray()
    
with TiffFile(os.path.join(data_dir, 'test-labels.tif')) as tif:
    test_label = tif.asarray()

M = max(np.max(train_img), np.max(eval_img), np.max(test_img))
m = min(np.min(train_img), np.min(eval_img), np.min(test_img))
train_img_norm = (train_img - m) / (M - m)
eval_img_norm = (eval_img - m) / (M - m)

In [6]:
# Custom Dataset class
class CustomDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        label = torch.from_numpy(label.astype(np.int64)).clone()
        label = torch.nn.functional.one_hot(label.long(), num_classes=7)
        label = label.to(torch.float32)
        label = label.permute(2, 0, 1)
        label = transforms.Resize((224, 224))(label)

        if self.transform:
            image = self.transform(image)

        return image, label

# FCN-ResNet50 model用のtransform
transform = transforms.Compose([
    # transforms.Resize((224, 224)), 
    transforms.ToTensor(),
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], 
    #                      std=[0.229, 0.224, 0.225])
])


# Creating the datasets
train_dataset = CustomDataset(train_img_norm, train_label, transform=transform)
eval_dataset = CustomDataset(eval_img_norm, eval_label, transform=transform)

# DataLoader creation
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
eval_loader = DataLoader(eval_dataset, batch_size=2, shuffle=False)

In [4]:
weights = FCN_ResNet50_Weights.DEFAULT
model = fcn_resnet50(weights=weights, num_classes=7)
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
# device = 'cpu'
model.to(device)
model.eval()

ValueError: The parameter 'num_classes' expected value 21 but got 7 instead.

In [ ]:
# 分類クラス数を引数として受け取るのに、21以外対応していない

In [27]:
print(next(model.parameters()).is_mps)

True


In [24]:
# 内部のバグで、summaryが使えない
summary(model, (3, 224, 224))

AttributeError: 'collections.OrderedDict' object has no attribute 'size'

In [36]:
img = train_img[0]
img

array([[15183, 14960, 14124, ...,  8637,  8298,  9734],
       [15088, 14882, 15025, ...,  7717,  7548,  8115],
       [15085, 14925, 14265, ...,  8987,  9257,  8946],
       ...,
       [11024, 11678, 11054, ...,  5606,  6552,  6630],
       [ 9074,  9459,  9957, ...,  5801,  7308,  7586],
       [10355,  9609,  8369, ...,  6613,  5735,  5393]], dtype=int16)

In [38]:
weights.transforms()

SemanticSegmentation(
    resize_size=[520]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BILINEAR
)

In [37]:
preprocess = weights.transforms()
preprocess = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
batch = preprocess(img)
batch

TypeError: Unexpected type <class 'numpy.ndarray'>

## segmentation_models_pytorchを使う

In [1]:
import os
from tifffile import TiffFile
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torchvision.models import vgg16_bn
import torch.optim as optim
from tqdm import tqdm
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
# from torchsummary import summary

In [2]:
data_dir = '../platelet_data'

with TiffFile(os.path.join(data_dir, 'train-images.tif')) as tif:
    train_img = tif.asarray()
    
with TiffFile(os.path.join(data_dir, 'train-labels.tif')) as tif:
    train_label = tif.asarray()
    
with TiffFile(os.path.join(data_dir, 'eval-images.tif')) as tif:
    eval_img = tif.asarray()
    
with TiffFile(os.path.join(data_dir, 'eval-labels.tif')) as tif:
    eval_label = tif.asarray()
    
with TiffFile(os.path.join(data_dir, 'test-images.tif')) as tif:
    test_img = tif.asarray()
    
with TiffFile(os.path.join(data_dir, 'test-labels.tif')) as tif:
    test_label = tif.asarray()

In [3]:
M = max(np.max(train_img), np.max(eval_img), np.max(test_img))
m = min(np.min(train_img), np.min(eval_img), np.min(test_img))
train_img_norm = (train_img - m) / (M - m)
eval_img_norm = (eval_img - m) / (M - m)

In [7]:
# Custom Dataset class
class CustomDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        label = torch.from_numpy(label.astype(np.int64)).clone()
        label = torch.nn.functional.one_hot(label.long(), num_classes=7)
        label = label.to(torch.float32)
        label = label.permute(2, 0, 1)
        # label = transforms.Resize((224, 224))(label)

        if self.transform:
            image = self.transform(image)
            image = image.to(torch.float32)

        return image, label

# FCN-ResNet50 model用のtransform
transform = transforms.Compose([
    transforms.ToTensor(),
    # transforms.Resize((224, 224)), 
    # transforms.Normalize(mean=[0.485, 0.456, 0.406], 
    #                      std=[0.229, 0.224, 0.225])
])


# Creating the datasets
train_dataset = CustomDataset(train_img_norm, train_label, transform=transform)
eval_dataset = CustomDataset(eval_img_norm, eval_label, transform=transform)

# DataLoader creation
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
eval_loader = DataLoader(eval_dataset, batch_size=2, shuffle=False)

### モデル1

In [8]:
# segmentation_models_pytorchモジュールの読み込み

import segmentation_models_pytorch as smp
import torch.nn.functional as F
import pytorch_lightning as pl
# from pytorch_lightning.metrics.functional import accuracy


# UNetの構築

class Net_resnet18(pl.LightningModule):
    def __init__(self, in_channels=1, n_classes=7):
        super().__init__()
        
        self.unet = smp.Unet("resnet18", in_channels=in_channels, classes=n_classes, encoder_weights="imagenet")
        
    def forward(self, x):
        return self.unet(x)
    
    def training_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        # loss = F.cross_entropy(y, t)
        loss = F.binary_cross_entropy_with_logits(y, t)
        self.log("train_loss", loss, on_step=True, on_epoch=True)
        # self.log("train_acc", accuracy(y.softmax(dim=1), t), on_step=True, on_epoch=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, t = batch
        y = self(x)
        # loss = F.cross_entropy(y, t)
        loss = F.binary_cross_entropy_with_logits(y, t)
        self.log("val_loss", loss, on_step=True, on_epoch=True)
        # self.log("val_acc", accuracy(y.softmax(dim=1), t), on_step=True, on_epoch=True)
        return loss
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters())
        


In [9]:
pl.seed_everything(0)
net_resnet18 = Net_resnet18()
trainer = pl.Trainer(max_epochs=3, deterministic=False)
trainer.fit(net_resnet18, train_loader, eval_loader)

Seed set to 0
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name | Type | Params
------------------------------
0 | unet | Unet | 14.3 M
------------------------------
14.3 M    Trainable params
0         Non-trainable params
14.3 M    Total params
57.291    Total estimated model params size (MB)


/Users/imyu/anaconda3/envs/seg_new/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
/Users/imyu/anaconda3/envs/seg_new/lib/python3.11/site-packages/pytorch_lightning/loops/fit_loop.py:293: The number of training batches (13) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 2: 100%|██████████| 13/13 [00:21<00:00,  0.61it/s, v_num=7]

`Trainer.fit` stopped: `max_epochs=3` reached.


Epoch 2: 100%|██████████| 13/13 [00:21<00:00,  0.60it/s, v_num=7]


In [13]:
from torchmetrics.functional import jaccard_index

scores = []

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
# device = "cpu"

net_resnet18.eval()
with torch.no_grad():
    for i, data in enumerate(eval_loader):
        inputs, labels = data
        inputs = inputs.float().to(device)
        labels = labels.float().to(device)
        outputs = net_resnet18(inputs)
        print(jaccard_index(outputs, labels, num_classes=7, task="multiclass"))
        scores.append(jaccard_index(outputs, labels, num_classes=7, task="multiclass").to("cpu").numpy())
        # print(jaccard_index(labels, labels, num_classes=7, task="multiclass"))
        
print(np.mean(scores))



RuntimeError: Input type (MPSFloatType) and weight type (torch.FloatTensor) should be the same

### モデル2

In [4]:
import segmentation_models_pytorch as smp

# パラメーター
ENCODER = "efficientnet-b4"
ENCODER_WEIGHTS = "imagenet"
ACTIVATION = "softmax2d"
CLASS_NUM = 7 # segmentationの正解ラベル数

# モデル定義
model = smp.UnetPlusPlus(
    encoder_name=ENCODER, 
    encoder_weights=ENCODER_WEIGHTS, 
    classes=CLASS_NUM, 
    activation=ACTIVATION,
)

/Users/imyu/anaconda3/envs/seg_new/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
# 学習時の設定
DEVICE = "cuda"
BATCH_SIZE = 1
# loss = smp.utils.losses.DiceLoss()
loss = smp.losses.JaccardLoss(mode='multiclass')
# metrics = [smp.utils.metrics.IoU(threshold=0.5)]
metrics = [smp.metrics.iou_score()]
optimizer = torch.optim.Adam([dict(params=model.parameters(), lr=0.0001)])

train_epoch = smp.utils.train.TrainEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    optimizer=optimizer,
    device=DEVICE,
    verbose=True,
)

valid_epoch = smp.utils.train.ValidEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    device=DEVICE,
    verbose=True,
)

TypeError: iou_score() missing 4 required positional arguments: 'tp', 'fp', 'fn', and 'tn'